Package Installations

In [1]:
!pip install kaggle

In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 35.4 MB/s 
     |████████████████████████████████| 596 kB 35.3 MB/s 
     |████████████████████████████████| 77 kB 2.9 MB/s 
     |████████████████████████████████| 895 kB 36.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install -U torchtext==0.8.0

     |████████████████████████████████| 6.9 MB 3.9 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [4]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import random
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm.notebook import tqdm
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import transformers
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
seed_val = 500
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Kaggle Dataset

In [7]:
# Create the kaggle directory and read the uploaded kaggle.json file
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
# Download dataset
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 71% 29.0M/41.0M [00:00<00:00, 68.8MB/s]
100% 41.0M/41.0M [00:00<00:00, 70.8MB/s]


In [10]:
# Unzip folder in Colab content folder
!unzip /content/fake-and-real-news-dataset.zip

Archive:  /content/fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [11]:
df1 = pd.read_csv("Fake.csv")
df2 = pd.read_csv("True.csv")
df1["y"] = ["False"]*len(df1)
df2["y"] = ["True"]*len(df2)

In [12]:
frames = [df1, df2]
corpus = pd.concat(frames, ignore_index=True)

In [13]:
corpus["x"] = corpus["title"]

In [14]:
y_encoder = LabelEncoder()
corpus['y'] = y_encoder.fit_transform(corpus['y'])

In [15]:
corpus = corpus[["x", "y"]]

In [16]:
corpus.drop_duplicates(inplace=True)
corpus

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0
...,...,...
44892,North Korea shipments to Syria chemical arms a...,1
44894,LexisNexis withdrew two products from Chinese ...,1
44895,Minsk cultural hub becomes haven from authorities,1
44896,Vatican upbeat on possibility of Pope Francis ...,1


**Reddit Dataset**

In [17]:
reddit_df = pd.read_csv('/content/news_posts.csv')
reddit_df

,Unnamed: 0,Title,Post Text,ID,Score,Total Comments,Post URL
0,0,People Are Accusing Robinhood Of Stealing From...,NaN,l7afyx,181017,4408,https://www.buzzfeednews.com/article/clarissaj...
1,1,US Military Could Lose Space Force Trademark t...,NaN,gyzw2p,129257,2844,https://www.cbr.com/us-military-lose-space-for...
2,2,White House threatens to fire anyone who tries...,NaN,jrskag,126468,4141,https://americanindependent.com/white-house-th...
3,3,Meta's threat to close down Facebook and Insta...,NaN,so0ree,126258,3786,https://www.cityam.com/metas-threat-to-close-d...
4,4,Don't eat or inject yourself with disinfectant...,NaN,g6zci5,125437,7024,https://www.cnn.com/world/live-news/coronaviru...
...,...,...,...,...,...,...,...
4898,987,Colorado pizza delivery driver saves mans life...,NaN,3b37gf,24221,898,http://www.postindependent.com/news/16943384-1...
4899,988,"Utah Woman Donates Over 1,200 Handmade Toys to...",NaN,90pbng,24216,888,https://www.insideedition.com/utah-woman-donat...
4900,989,Year of the Tiger marks increase in tiger popu...,NaN,sinyqg,24197,205,https://democratic-europe.eu/2022/02/01/%ef%bf...
4901,990,Florida cops deliver dresses made by a 99-year...,NaN,8q8njr,24164,183,http://www.miamiherald.com/news/state/florida/...


In [18]:
reddit_df['x'] = reddit_df['Title']

In [19]:
reddit_df['y'] = ["True"]*4903
reddit_df['y'] = y_encoder.transform(reddit_df['y'])

In [20]:
reddit_df = reddit_df[["x", "y"]]
reddit_df.drop_duplicates(inplace=True)
reddit_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,People Are Accusing Robinhood Of Stealing From...,1
1,US Military Could Lose Space Force Trademark t...,1
2,White House threatens to fire anyone who tries...,1
3,Meta's threat to close down Facebook and Insta...,1
4,Don't eat or inject yourself with disinfectant...,1
...,...,...
4898,Colorado pizza delivery driver saves mans life...,1
4899,"Utah Woman Donates Over 1,200 Handmade Toys to...",1
4900,Year of the Tiger marks increase in tiger popu...,1
4901,Florida cops deliver dresses made by a 99-year...,1


**Twitter Dataset**

In [21]:
twitter_real_df = pd.read_csv('/content/real_news_tweets.csv')
twitter_real_df

,Unnamed: 0,Datetime,Tweet Id,Text,Username
0,0,2022-04-19 08:35:25+00:00,1516334641610989571,Half-a-million Ukrainians have fled to the Lvi...,AP
1,1,2022-04-19 07:42:46+00:00,1516321393281015809,Afghan police say explosions targeting educati...,AP
2,2,2022-04-19 07:13:55+00:00,1516314133666349058,Russian forces have attacked along a broad fro...,AP
3,3,2022-04-19 06:40:38+00:00,1516305757674819584,There's stark public divide in Iran over Russi...,AP
4,4,2022-04-19 06:26:26+00:00,1516302180772364288,There's stark public divide in Iran over Russi...,AP
...,...,...,...,...,...
4995,4995,2022-02-24 00:13:22+00:00,1496639353820430337,The soldiers on the front line of the Ukraine-...,BBCWorld
4996,4996,2022-02-23 23:45:08+00:00,1496632246459158531,What's Putin's next move?,BBCWorld
4997,4997,2022-02-23 23:29:06+00:00,1496628212880875520,"Virgin Hyperloop, a futuristic train service e...",BBCWorld
4998,4998,2022-02-23 23:14:06+00:00,1496624437243879429,Metaverse app allows kids into virtual strip c...,BBCWorld


In [22]:
twitter_fake_df = pd.read_csv('/content/fake_news_tweets.csv')
twitter_fake_df

,Unnamed: 0,Datetime,Tweet Id,Text,Username
0,0,2022-04-19 08:30:02+00:00,1516333287744102405,Grandpa Surprisingly Willing To Talk About Man...,TheOnion
1,1,2022-04-19 06:30:01+00:00,1516303084628099076,Nation Attempts To Fall Asleep By Doing Little...,TheOnion
2,2,2022-04-19 05:00:01+00:00,1516280436334473221,"When it comes to pip blips, Megan Fox takes th...",TheOnion
3,3,2022-04-19 04:30:01+00:00,1516272886314672130,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,TheOnion
4,4,2022-04-19 03:15:00+00:00,1516254005869096968,The Onion: Now on another part of your phone. ...,TheOnion
...,...,...,...,...,...
4995,4995,2021-10-02 14:00:00+00:00,1444301140703780868,Boss Warns All New Employees He Will Fire Them...,TheBabylonBee
4996,4996,2021-10-02 13:00:02+00:00,1444286051204403205,10 Tips For Making Sure Your Kids Are Terrifie...,TheBabylonBee
4997,4997,2021-10-02 12:21:32+00:00,1444276362865479696,Biden Says We Can Afford $3.5T Bill Because Ch...,TheBabylonBee
4998,4998,2021-10-02 02:00:00+00:00,1444119947274334211,Fly That Landed On Mike Pence During Debate Re...,TheBabylonBee


In [23]:
twitter_real_df['x'] = twitter_real_df['Text']
twitter_fake_df['x'] = twitter_fake_df['Text']

In [24]:
twitter_real_df['y'] = ['True'] * 5000
twitter_real_df['y'] = y_encoder.transform(twitter_real_df['y'])

In [25]:
twitter_fake_df['y'] = ['False'] * 5000
twitter_fake_df['y'] = y_encoder.transform(twitter_fake_df['y'])

In [26]:
twitter_real_df = twitter_real_df[["x", "y"]]
twitter_real_df.drop_duplicates(inplace=True)
twitter_real_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Half-a-million Ukrainians have fled to the Lvi...,1
1,Afghan police say explosions targeting educati...,1
2,Russian forces have attacked along a broad fro...,1
3,There's stark public divide in Iran over Russi...,1
4,There's stark public divide in Iran over Russi...,1
...,...,...
4995,The soldiers on the front line of the Ukraine-...,1
4996,What's Putin's next move?,1
4997,"Virgin Hyperloop, a futuristic train service e...",1
4998,Metaverse app allows kids into virtual strip c...,1


In [27]:
twitter_fake_df = twitter_fake_df[["x", "y"]]
twitter_fake_df.drop_duplicates(inplace=True)
twitter_fake_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Grandpa Surprisingly Willing To Talk About Man...,0
1,Nation Attempts To Fall Asleep By Doing Little...,0
2,"When it comes to pip blips, Megan Fox takes th...",0
3,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,0
4,The Onion: Now on another part of your phone. ...,0
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


In [28]:
from sklearn import model_selection
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [29]:
frame1 = [twitter_real_df, twitter_fake_df]
corpus_tweet = pd.concat(frame1)
corpus_tweet

,x,y
0,Half-a-million Ukrainians have fled to the Lvi...,1
1,Afghan police say explosions targeting educati...,1
2,Russian forces have attacked along a broad fro...,1
3,There's stark public divide in Iran over Russi...,1
4,There's stark public divide in Iran over Russi...,1
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


In [30]:
#Convert text to lowercase
corpus['x'] = [text.lower() for text in corpus['x']]

#Tokenization
corpus['x'] = [word_tokenize(text) for text in corpus['x']]

#WordNetLemmatizer
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [31]:
membersProcessed = 0
for idx, text in enumerate(corpus['x']):
  finalWords = []
  word_net_lemmatizer = WordNetLemmatizer()
  set_stop = set(stopwords.words('english'))
  iterate = pos_tag(text)
  [finalWords.append(word_net_lemmatizer.lemmatize(word, tag_map[tag[0]])) for word, tag in iterate if word not in set_stop and word.isalpha()]
  corpus.loc[idx, 'finalText'] = str(finalWords)
  membersProcessed+=1
  print('Progress: {}/{} members processed'.format(membersProcessed, len(corpus)))

Streaming output truncated to the last 5000 lines.
Progress: 33730/44416 members processed
Progress: 33731/44416 members processed
Progress: 33732/44416 members processed
Progress: 33733/44416 members processed
Progress: 33734/44416 members processed
Progress: 33735/44416 members processed
Progress: 33736/44416 members processed
Progress: 33737/44416 members processed
Progress: 33738/44416 members processed
Progress: 33739/44416 members processed
Progress: 33740/44416 members processed
Progress: 33741/44416 members processed
Progress: 33742/44416 members processed
Progress: 33743/44416 members processed
Progress: 33744/44416 members processed
Progress: 33745/44416 members processed
Progress: 33746/44416 members processed
Progress: 33747/44416 members processed
Progress: 33748/44416 members processed
Progress: 33749/44416 members processed
Progress: 33750/44416 members processed
Progress: 33751/44416 members processed
Progress: 33752/44416 members processed
Progress: 33753/44416 members

In [32]:
#Convert text to lowercase
reddit_df['x'] = [text.lower() for text in reddit_df['x']]

#Tokenization
reddit_df['x'] = [word_tokenize(text) for text in reddit_df['x']]

#WordNetLemmatizer
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [33]:
membersProcessed = 0
for idx, text in enumerate(reddit_df['x']):
  finalWords = []
  word_net_lemmatizer = WordNetLemmatizer()
  set_stop = set(stopwords.words('english'))
  iterate = pos_tag(text)
  [finalWords.append(word_net_lemmatizer.lemmatize(word, tag_map[tag[0]])) for word, tag in iterate if word not in set_stop and word.isalpha()]
  reddit_df.loc[idx, 'finalText1'] = str(finalWords)
  membersProcessed+=1
  print('Progress: {}/{} members processed'.format(membersProcessed, len(reddit_df)))

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Progress: 1/4903 members processed
Progress: 2/4903 members processed
Progress: 3/4903 members processed
Progress: 4/4903 members processed
Progress: 5/4903 members processed
Progress: 6/4903 members processed
Progress: 7/4903 members processed
Progress: 8/4903 members processed
Progress: 9/4903 members processed
Progress: 10/4903 members processed
Progress: 11/4903 members processed
Progress: 12/4903 members processed
Progress: 13/4903 members processed
Progress: 14/4903 members processed
Progress: 15/4903 members processed
Progress: 16/4903 members processed
Progress: 17/4903 members processed
Progress: 18/4903 members processed
Progress: 19/4903 members processed
Progress: 20/4903 members processed
Progress: 21/4903 members processed
Progress: 22/4903 members processed
Progress: 23/4903 members processed
Progress: 24/4903 members processed
Progress: 25/4903 members processed
Progress: 26/4903 members processed
Progress: 27/4903 members processed
Progress: 28/4903 members processed
P

In [34]:
corpus

,x,y,finalText
0,"[donald, trump, sends, out, embarrassing, new,...",0.0,"['donald', 'trump', 'send', 'embarrass', 'new'..."
1,"[drunk, bragging, trump, staffer, started, rus...",0.0,"['drunk', 'brag', 'trump', 'staffer', 'start',..."
2,"[sheriff, david, clarke, becomes, an, internet...",0.0,"['sheriff', 'david', 'clarke', 'become', 'inte..."
3,"[trump, is, so, obsessed, he, even, has, obama...",0.0,"['trump', 'obsessed', 'even', 'obama', 'name',..."
4,"[pope, francis, just, called, out, donald, tru...",0.0,"['pope', 'francis', 'call', 'donald', 'trump',..."
...,...,...,...
38654,NaN,NaN,"['photo', 'north', 'korea', 'signal', 'powerfu..."
38664,NaN,NaN,"['thailand', 'oust', 'pm', 'yingluck', 'flee',..."
38692,NaN,NaN,"['britain', 'study', 'effect', 'foreign', 'stu..."
38706,NaN,NaN,"['trump', 'must', 'respect', 'president', 'say..."


In [35]:
corpus = corpus.dropna()

In [36]:
corpus

,x,y,finalText
0,"[donald, trump, sends, out, embarrassing, new,...",0.0,"['donald', 'trump', 'send', 'embarrass', 'new'..."
1,"[drunk, bragging, trump, staffer, started, rus...",0.0,"['drunk', 'brag', 'trump', 'staffer', 'start',..."
2,"[sheriff, david, clarke, becomes, an, internet...",0.0,"['sheriff', 'david', 'clarke', 'become', 'inte..."
3,"[trump, is, so, obsessed, he, even, has, obama...",0.0,"['trump', 'obsessed', 'even', 'obama', 'name',..."
4,"[pope, francis, just, called, out, donald, tru...",0.0,"['pope', 'francis', 'call', 'donald', 'trump',..."
...,...,...,...
38724,"[war, crimes, court, backs, burundi, investiga...",1.0,"['north', 'korea', 'shipment', 'syria', 'chemi..."
38725,"[uganda, 's, president, boosts, military, unit...",1.0,"['lexisnexis', 'withdraw', 'two', 'product', '..."
38726,"[despite, nato, pledge, to, increase, afghan, ...",1.0,"['minsk', 'cultural', 'hub', 'becomes', 'autho..."
38727,"[syria, declares, victory, over, islamic, state]",1.0,"['vatican', 'upbeat', 'possibility', 'pope', '..."


In [37]:
reddit_df

,x,y,finalText1
0,"[people, are, accusing, robinhood, of, stealin...",1,"['people', 'accuse', 'robinhood', 'steal', 'po..."
1,"[us, military, could, lose, space, force, trad...",1,"['u', 'military', 'could', 'lose', 'space', 'f..."
2,"[white, house, threatens, to, fire, anyone, wh...",1,"['white', 'house', 'threaten', 'fire', 'anyone..."
3,"[meta, 's, threat, to, close, down, facebook, ...",1,"['meta', 'threat', 'close', 'facebook', 'insta..."
4,"[do, n't, eat, or, inject, yourself, with, dis...",1,"['eat', 'inject', 'disinfectant', 'warn', 'fda..."
...,...,...,...
4898,"[colorado, pizza, delivery, driver, saves, man...",1,"['colorado', 'pizza', 'delivery', 'driver', 's..."
4899,"[utah, woman, donates, over, 1,200, handmade, ...",1,"['utah', 'woman', 'donate', 'handmade', 'toy',..."
4900,"[year, of, the, tiger, marks, increase, in, ti...",1,"['year', 'tiger', 'mark', 'increase', 'tiger',..."
4901,"[florida, cops, deliver, dresses, made, by, a,...",1,"['florida', 'cop', 'deliver', 'dress', 'make',..."


In [38]:
reddit_df = reddit_df.dropna()
reddit_df

,x,y,finalText1
0,"[people, are, accusing, robinhood, of, stealin...",1,"['people', 'accuse', 'robinhood', 'steal', 'po..."
1,"[us, military, could, lose, space, force, trad...",1,"['u', 'military', 'could', 'lose', 'space', 'f..."
2,"[white, house, threatens, to, fire, anyone, wh...",1,"['white', 'house', 'threaten', 'fire', 'anyone..."
3,"[meta, 's, threat, to, close, down, facebook, ...",1,"['meta', 'threat', 'close', 'facebook', 'insta..."
4,"[do, n't, eat, or, inject, yourself, with, dis...",1,"['eat', 'inject', 'disinfectant', 'warn', 'fda..."
...,...,...,...
4898,"[colorado, pizza, delivery, driver, saves, man...",1,"['colorado', 'pizza', 'delivery', 'driver', 's..."
4899,"[utah, woman, donates, over, 1,200, handmade, ...",1,"['utah', 'woman', 'donate', 'handmade', 'toy',..."
4900,"[year, of, the, tiger, marks, increase, in, ti...",1,"['year', 'tiger', 'mark', 'increase', 'tiger',..."
4901,"[florida, cops, deliver, dresses, made, by, a,...",1,"['florida', 'cop', 'deliver', 'dress', 'make',..."


In [39]:
#Convert text to lowercase
corpus_tweet['x'] = [text.lower() for text in corpus_tweet['x']]

#Tokenization
corpus_tweet['x'] = [word_tokenize(text) for text in corpus_tweet['x']]

#WordNetLemmatizer
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [40]:
membersProcessed = 0
for idx, text in enumerate(corpus_tweet['x']):
  finalWords = []
  word_net_lemmatizer = WordNetLemmatizer()
  set_stop = set(stopwords.words('english'))
  iterate = pos_tag(text)
  [finalWords.append(word_net_lemmatizer.lemmatize(word, tag_map[tag[0]])) for word, tag in iterate if word not in set_stop and word.isalpha()]
  corpus_tweet.loc[idx, 'finalText1'] = str(finalWords)
  membersProcessed+=1
  print('Progress: {}/{} members processed'.format(membersProcessed, len(corpus_tweet)))

Streaming output truncated to the last 5000 lines.
Progress: 3918/8948 members processed
Progress: 3919/8948 members processed
Progress: 3920/8948 members processed
Progress: 3921/8948 members processed
Progress: 3922/8948 members processed
Progress: 3923/8948 members processed
Progress: 3924/8948 members processed
Progress: 3925/8948 members processed
Progress: 3926/8948 members processed
Progress: 3927/8948 members processed
Progress: 3928/8948 members processed
Progress: 3929/8948 members processed
Progress: 3930/8948 members processed
Progress: 3931/8948 members processed
Progress: 3932/8948 members processed
Progress: 3933/8948 members processed
Progress: 3934/8948 members processed
Progress: 3935/8948 members processed
Progress: 3936/8948 members processed
Progress: 3937/8948 members processed
Progress: 3938/8949 members processed
Progress: 3939/8949 members processed
Progress: 3940/8949 members processed
Progress: 3941/8949 members processed
Progress: 3942/8949 members processed

In [41]:
corpus_tweet = corpus_tweet.dropna()
corpus_tweet

,x,y,finalText1
0,"[half-a-million, ukrainians, have, fled, to, t...",1.0,"['ukrainian', 'flee', 'lviv', 'region', 'weste..."
1,"[afghan, police, say, explosions, targeting, e...",1.0,"['afghan', 'police', 'say', 'explosion', 'targ..."
2,"[russian, forces, have, attacked, along, a, br...",1.0,"['russian', 'force', 'attack', 'along', 'broad..."
3,"[there, 's, stark, public, divide, in, iran, o...",1.0,"['stark', 'public', 'divide', 'iran', 'russia'..."
4,"[there, 's, stark, public, divide, in, iran, o...",1.0,"['stark', 'public', 'divide', 'iran', 'russia'..."
...,...,...,...
4995,"[boss, warns, all, new, employees, he, will, f...",0.0,"['wish', 'could', 'go', 'psychotic', 'breakdow..."
4996,"[10, tips, for, making, sure, your, kids, are,...",0.0,"['bad', 'part', 'tiktok', 'people', 'access', ..."
4997,"[biden, says, we, can, afford, $, 3.5t, bill, ...",0.0,"['absolutely', 'know', 'take', 'grant']"
4998,"[fly, that, landed, on, mike, pence, during, d...",0.0,"['epa', 'hop', 'streamline', 'ecosystem', 'hos..."


**Naive Bayes & SVM**

In [42]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=10)

In [43]:
X_train = corpus['finalText']
y_train = corpus['y']

In [47]:
X_test = pd.concat([reddit_df['finalText1'], corpus_tweet['finalText1']])
y_test = pd.concat([reddit_df['y'], corpus_tweet['y']])

In [48]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [49]:
acc_ls = []
TFIDF_vect = TfidfVectorizer(max_features=5000)
x_train_tfidf = TFIDF_vect.fit_transform(X_train)
x_test_tfidf = TFIDF_vect.transform(X_test)
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train_tfidf,y_train)
predictions_NB = Naive.predict(x_test_tfidf)
acc = accuracy_score(predictions_NB, y_test)
acc_ls.append(acc)
  
print("Naive Bayes Accuracy Score " + " -> ",acc*100)
print("Mean Accuracy : ", sum(acc_ls)*100/len(acc_ls))

Naive Bayes Accuracy Score  ->  45.20260492040521
Mean Accuracy :  45.20260492040521


In [50]:
acc_ls = []
TFIDF_vect = TfidfVectorizer(max_features=5000)
x_train_tfidf = TFIDF_vect.fit_transform(X_train)
x_test_tfidf = TFIDF_vect.transform(X_test)
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train_tfidf, y_train)
predictions_SVM = SVM.predict(x_test_tfidf)
acc = accuracy_score(predictions_SVM, y_test)
acc_ls.append(acc)
print("SVM Accuracy Score " + " -> ",acc*100)
print("Mean Accuracy : ", sum(acc_ls)*100/len(acc_ls))

SVM Accuracy Score  ->  49.34876989869754
Mean Accuracy :  49.34876989869754
